# BERT를 이용한 한국어 텍스트 분류(허깅페이스 많이 활용)

## Install Hugging Face Transformer

In [ ]:
# Hugging Face의 트랜스포머 모델을 설치
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.4 MB/s eta 0:00:00


In [ ]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.7/270.7 kB 4.4 MB/s eta 0:00:00


## 선학습 모델 설정

- https://huggingface.co/models?library=pytorch&pipeline_tag=fill-mask&sort=downloads&search=kc

In [ ]:
# 아래 선학습 파라미터는 beomi/kcbert-large로 학습한 것

# pretrained_mode_name = "beomi/kcbert-base"
pretrained_model_name = "beomi/kcbert-large"

In [ ]:
from transformers import AutoTokenizer
from transformers import BertForSequenceClassification
from transformers import get_linear_schedule_with_warmup

import torch
import pandas as pd
import numpy as np
import random
import time
import datetime
import re

from tqdm import tqdm

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [ ]:
!git clone https://github.com/e9t/nsmc.git

Cloning into 'nsmc'...
remote: Enumerating objects: 14763, done.
remote: Counting objects: 100% (14762/14762), done.
remote: Compressing objects: 100% (13012/13012), done.
remote: Total 14763 (delta 1748), reused 14762 (delta 1748), pack-reused 1
Receiving objects: 100% (14763/14763), 56.19 MiB | 23.60 MiB/s, done.
Resolving deltas: 100% (1748/1748), done.
Updating files: 100% (14737/14737), done.


In [ ]:
# 훈련셋과 테스트셋 데이터 로드
# 편의상 판다스 사용
train_df = pd.read_csv("nsmc/ratings_train.txt", sep='\t')
test_df = pd.read_csv("nsmc/ratings_test.txt", sep='\t')

print(train_df.shape)
print(test_df.shape)

(150000, 3)
(50000, 3)


In [ ]:
# info()를 찍어보면 document에 5개 null이 있다는 것을 알 수 있음
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [ ]:
train_df.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [ ]:
train_df[train_df['document'].isnull()]

,id,document,label
25857,2172111,NaN,1
55737,6369843,NaN,1
110014,1034280,NaN,0
126782,5942978,NaN,0
140721,1034283,NaN,0


In [ ]:
test_df[test_df['document'].isnull()]

,id,document,label
5746,402110,NaN,1
7899,5026896,NaN,0
27097,511097,NaN,1


In [ ]:
# null 데이터 확인
# 원래 데이터에서는 ''인 빈문자열인데 pandas가 로딩하면서 nan으로 바꾼것
train_df.loc[25857,'document']

nan

In [ ]:
# 그냥 dropna()
train_df = train_df.dropna()
test_df = test_df.dropna()

In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 149995 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        149995 non-null  int64 
 1   document  149995 non-null  object
 2   label     149995 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.6+ MB


In [ ]:
train_df.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [ ]:
# 허깅페이스 데이터 셋
from datasets import Dataset

ds_hf = Dataset.from_pandas(train_df.loc[:,'document':'label'])

In [ ]:
ds_hf

Dataset({
    features: ['document', 'label', '__index_level_0__'],
    num_rows: 149995
})

In [ ]:
# 굳이 '__index_level_0__' 컬럼 지우지 않음
# ds_hf = ds_hf.remove_columns(['__index_level_0__'])
# ds_hf

In [ ]:
ds_hf[:3]

{'document': ['아 더빙.. 진짜 짜증나네요 목소리',
  '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나',
  '너무재밓었다그래서보는것을추천한다'],
 'label': [0, 1, 0],
 '__index_level_0__': [0, 1, 2]}

## 토큰화

- 허깅페이스에서 제공하는 BertTokenizer를 사용하여 토큰화 시도

- 형태소 분석같은 언어 종속적인 사전 지식을 사용하지 않음

- WordPiece 알고리즘 적용하여 분절 시도

- 도움말: https://huggingface.co/docs/tokenizers/python/latest/

In [ ]:
# 토크나이저 초기화
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/672 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/250k [00:00<?, ?B/s]

In [ ]:
# 토크나이저의 중요 프로퍼티 3개는 중요하니까 꼭 암기!
(
    tokenizer.vocab_size, # 단어장 길이
    tokenizer.model_max_length, # 입력 시퀀스 길이
    tokenizer.model_input_names # 모델이 입력받는 입력의 이름들
)

(30000, 300, ['input_ids', 'token_type_ids', 'attention_mask'])

In [ ]:
# 토큰화 실험
output = tokenizer.encode("Hello, y'all! How are you 😁 ?")

# 숫자로 인코딩된 결과
# tokenizer("Hello, y'all! How are you 😁 ?") 로 실행한 결과와 비교 해볼 것
print(output)

[2, 41, 4226, 18993, 4404, 15, 90, 10, 66, 18993, 5, 41, 23468, 66, 17107, 23380, 3850, 32, 3]


In [ ]:
# 인코딩된 숫자를 토큰화
print( tokenizer.convert_ids_to_tokens(output) )

['[CLS]', 'H', '##e', '##ll', '##o', ',', 'y', "'", 'a', '##ll', '!', 'H', '##ow', 'a', '##re', 'you', '😁', '?', '[SEP]']


In [ ]:
# 토큰화 직접 해보기

# 숫자를 토큰으로 바꾸는 사전
idx_to_tkn = { v:k for k, v in tokenizer.vocab.items()}

# 토큰화된 index를 토큰으로 변환하는 함수
def idx2tokens(encoding):
    return [ tuple(map(idx_to_tkn.get, output)) for output in encoding ]

print( idx2tokens([output]) )

[('[CLS]', 'H', '##e', '##ll', '##o', ',', 'y', "'", 'a', '##ll', '!', 'H', '##ow', 'a', '##re', 'you', '😁', '?', '[SEP]')]


- BERT에 입력으로 시작과 끝에 [CLS], [SEP]가 들어가야 하므로 자동으로 해당 토큰을 추가하는 것을 확인할 수 있음

### NSMC에 시험적으로 토큰화 적용

In [ ]:
ds_hf[:5]

{'document': ['아 더빙.. 진짜 짜증나네요 목소리',
  '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나',
  '너무재밓었다그래서보는것을추천한다',
  '교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정',
  '사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다'],
 'label': [0, 1, 0, 0, 1],
 '__index_level_0__': [0, 1, 2, 3, 4]}

In [ ]:
# BERT의 입력 형식에 맞게 변환
sentences_for_bert = [ "[CLS] " + doc + " [SEP]" for doc in ds_hf['document'][:5] ]
sentences_for_bert

['[CLS] 아 더빙.. 진짜 짜증나네요 목소리 [SEP]',
 '[CLS] 흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나 [SEP]',
 '[CLS] 너무재밓었다그래서보는것을추천한다 [SEP]',
 '[CLS] 교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정 [SEP]',
 '[CLS] 사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다 [SEP]']

In [ ]:
# tokenizer.tokenize()는 숫자로 인코딩하지 않고 바로 토큰화해서 보여줌
tokenizer.tokenize(sentences_for_bert[0])

['[CLS]', '아', '더', '##빙', '.', '.', '진짜', '짜증나네', '##요', '목소리', '[SEP]']

In [ ]:
# 토크나이저로 토큰화
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences_for_bert]

In [ ]:
print("토크나이저로 들어가는 토큰화 전 문장: ", sentences_for_bert[0])
print("토큰화 된 문장: ", tokenized_texts[0])
print(type(tokenized_texts))

토크나이저로 들어가는 토큰화 전 문장:  [CLS] 아 더빙.. 진짜 짜증나네요 목소리 [SEP]
토큰화 된 문장:  ['[CLS]', '아', '더', '##빙', '.', '.', '진짜', '짜증나네', '##요', '목소리', '[SEP]']
<class 'list'>


- split 만들기

In [ ]:
df_hf = ds_hf.train_test_split(test_size=0.2)
df_hf['valid'] = df_hf['test']
df_hf['test'] = Dataset.from_pandas(test_df.loc[:,'document':'label'])
df_hf

DatasetDict({
    train: Dataset({
        features: ['document', 'label', '__index_level_0__'],
        num_rows: 119996
    })
    test: Dataset({
        features: ['document', 'label', '__index_level_0__'],
        num_rows: 49997
    })
    valid: Dataset({
        features: ['document', 'label', '__index_level_0__'],
        num_rows: 29999
    })
})

- 데이터 셋의 모든 샘플을 숫자로 바꿈

In [ ]:
MAX_LENGTH = 128

def tokenize(samples):
    # 패딩하지 않음!
    # 패딩은 콜레이터에서 할 것
    return tokenizer(samples['document'],
                     truncation=True, max_length=MAX_LENGTH)


In [ ]:
ds_encoded = df_hf.map(tokenize, batched=True,
                       # 필요없는 컬럼
                       remove_columns=['document', '__index_level_0__'])

Map:   0%|          | 0/119996 [00:00<?, ? examples/s]

Map:   0%|          | 0/49997 [00:00<?, ? examples/s]

Map:   0%|          | 0/29999 [00:00<?, ? examples/s]

In [ ]:
ds_encoded

# DatasetDict({
#     train: Dataset({
#         features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
#         num_rows: 119996
#     })
#     test: Dataset({
#         features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
#         num_rows: 49997
#     })
#     valid: Dataset({
#         features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
#         num_rows: 29999
#     })
# })

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 119996
    })
    test: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 49997
    })
    valid: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 29999
    })
})

In [ ]:
# 패딩이 안되어있기 때문에 input_ids길이는 다 다름
[len(input_ids) for input_ids in ds_encoded['train'][:3]['input_ids']]

[9, 14, 14]

## 콜레이터

In [ ]:
from transformers import DataCollatorWithPadding

In [ ]:
data_collator = DataCollatorWithPadding(
    tokenizer=tokenizer,
    padding=True
)

In [ ]:
sample_for_collator = [ ds_encoded['train'][i] for i in range(0, 3) ]

In [ ]:
sample = data_collator(sample_for_collator)
sample

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'input_ids': tensor([[    2, 13365,  4126, 13199, 25253, 11212, 11071,  9376,     3,     0,
             0,     0,     0,     0],
        [    2, 26085,  4771,  4017,  3516, 11422, 14209,  4008, 15759, 16731,
         10212,  4020,    17,     3],
        [    2,  3297,  4358,  4598,  4009,  8117, 29987, 23661, 12401,  9376,
          4008, 20421,  8063,     3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'labels': tensor([1, 1, 1])}

## 모델

In [ ]:
model = BertForSequenceClassification.from_pretrained(pretrained_model_name,
                                                      num_labels=2)

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/kcbert-large and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### 포워드 테스트

In [ ]:
# 포워드 테스트
output = model(**sample)

In [ ]:
# 로스와 로짓이 포함되어 있음
output

# 로짓은 (3,2)

SequenceClassifierOutput(loss=tensor(0.5334, grad_fn=<NllLossBackward0>), logits=tensor([[-0.2975,  0.3195],
        [-0.3179,  0.0860],
        [ 0.0120,  0.0847]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

## 선학습 파라미터 로딩

- 여기서 파라미터를 로딩했으면 학습은 하지말고 바로 테스트로 넘어가기

In [ ]:
# !gdown --id 1PpxZVkC7CO2DvM0ycWi_UYmuH69eEdX8

In [ ]:
# model.load_state_dict(torch.load("nsmc-kcbert-large.pth"))
# model.to(device)

## 학습

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

In [ ]:
def compute_metrics(pred):
    # pred: EvalPrediction 객체
    # 이 객체는 HF Trainer가 학습을 할 때 Eval 시점에서 생성되고 이 함수로 넘겨지게 됨
    # transformers 소스에 이렇게 호출됨
    # metrics = self.compute_metrics(
    #                EvalPrediction(predictions=preds, label_ids=label_ids, inputs=inputs_ids)
    #          )
    labels = pred.label_ids # 정답
    # https://huggingface.co/docs/transformers/internal/trainer_utils
    preds = pred.predictions.argmax(-1) # (N,C), pred.predictions: ndarray

    # sklearn함수에 바로 집어 넣는 걸로 봐서 ndarray 같은데
    f1 = f1_score(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {'accuracy': acc, 'f1': f1}

In [ ]:
from transformers import Trainer, TrainingArguments

In [ ]:
epochs = 2
batch_size = 32
# total_steps = len(ds_encoded['train'])*epochs
# logging_steps = len(ds_encoded['train']) // batch_size
model_name = f"{pretrained_model_name}-finetuned-nsmc"

# 좀 길기는 한데 그냥 하이퍼파라미터 설정하는 것
training_args = TrainingArguments(
    ############################################################
    # Epoch, Learning Rate and Mini Batch Size
    ############################################################
    output_dir=model_name,
    num_train_epochs=epochs,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,

    ############################################################
    # Learning rate schedule
    ############################################################
    # https://huggingface.co/docs/transformers/main_classes/optimizer_schedules#transformers.get_linear_schedule_with_warmup
    lr_scheduler_type ='linear',
    warmup_steps=1500, # 1500 스탭까지 0부터 learning_rate 상승 이후 하강
    # 4.35에 없고 4.36부터 있는 옵션
    # lr_scheduler_kwargs = {
    #     'num_warmup_steps': 0,
    #     'num_training_steps': total_steps
    # },

    ############################################################
    # Eval. and Save
    ############################################################
    evaluation_strategy='steps',
    eval_steps=500, # 500스탭 마다 한번씩 eval
    save_strategy='steps', # eval하고 모델 저장
    load_best_model_at_end=True, # 학습 후 베스트 모델 로딩

    ############################################################
    # Logging
    ############################################################
    # 이 로깅 옵션이 명확하지 않음
    # 폴더에 파일이 하나 생기는데 같은 파일이 계속 logging_step마다 업데이트 됨
    # 텐서보드 파일 같음
    # logging_dir='./logs',
    # logging_strategy='steps',
    # log_level='info',
    # logging_first_step=True,
    # logging_steps=500,
    # disable_tqdm=False,
    # push_to_hub=True,
)

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# https://github.com/huggingface/transformers/issues/27631
# https://discuss.huggingface.co/t/how-to-show-the-learning-rate-during-training/13914/4
class MyTrainer(Trainer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def log(self, logs):
        logs["learning_rate"] = self._get_learning_rate()
        super().log(logs)


trainer = MyTrainer(
    model=model, args=training_args,
    # 콜레이터를 지정하지 않고 토크나이저만 지정하면
    # DataCollatorWithPadding을 기본 콜레이터로 사용함
    # 미리 준비한 콜레이터도 DataCollatorWithPadding이므로
    # 이 경우 꼭 콜레이터를 지정하지 않아도 학습이 잘 진행됨
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    train_dataset=ds_encoded['train'],
    eval_dataset=ds_encoded['valid'],
    tokenizer=tokenizer # 모델이 허브에 저장될때 같이 저장된다고 함
)

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy,F1,Rate
500,0.501300,0.318824,0.866829,0.866726,0.000007
1000,0.301600,0.287415,0.884196,0.884026,0.000013
1500,0.281600,0.277099,0.892630,0.892624,0.000020
2000,0.276200,0.263630,0.890263,0.890086,0.000018
2500,0.256400,0.250335,0.900097,0.900005,0.000017
3000,0.255700,0.232225,0.905097,0.905091,0.000015
3500,0.256500,0.230408,0.906564,0.906564,0.000013
4000,0.205500,0.248627,0.908997,0.908997,0.000012
4500,0.172400,0.243794,0.906630,0.906622,0.000010
5000,0.165000,0.239100,0.909297,0.909278,0.000008


TrainOutput(global_step=7500, training_loss=0.23060399373372395, metrics={'train_runtime': 4660.28, 'train_samples_per_second': 51.497, 'train_steps_per_second': 1.609, 'total_flos': 2.748186958745491e+16, 'train_loss': 0.23060399373372395, 'learning_rate': 0.0, 'epoch': 2.0})

## 테스트

In [ ]:
model.eval();

In [ ]:
# 허깅페이스 이용한 테스트 루프 어떻게 만들지?
# 이렇게 스플릿을 루프에 걸어서 돌릴 수도 있지만...
for d in ds_encoded['test']:
    print(d)
    break

{'label': 1, 'input_ids': [2, 352, 192, 3], 'token_type_ids': [0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1]}


In [ ]:
# 허깅페이스 데이터 셋을 데이터로더에 바로 입력할 수 있음
from torch.utils.data import DataLoader

test_loader =  DataLoader(ds_encoded['test'],
                          batch_size=batch_size,
                          collate_fn=data_collator)

test_loader_iter = iter(test_loader)

In [ ]:
batch = next(test_loader_iter)

# 실행할 때마다 다른 숫자들로 채워진 리스트가 나오면 콜레이터가 잘 작동하는 것
print([len(x) for x in batch['input_ids']])

[58, 58, 58, 58, 58, 58, 58, 58, 58, 58, 58, 58, 58, 58, 58, 58, 58, 58, 58, 58, 58, 58, 58, 58, 58, 58, 58, 58, 58, 58, 58, 58]


In [ ]:
corr = 0

with torch.no_grad():
    for i, data in enumerate(tqdm(test_loader)):
        X = data['input_ids'].to(device)
        mask = data['attention_mask'].to(device)
        y = data['labels'].to(device)

        score = model(X, attention_mask=mask).logits
        y_prob = torch.nn.functional.softmax(score, dim=-1)
        y_pred = torch.argmax(y_prob, dim=-1)
        corr += (y_pred == y).sum()

test_acc = corr / len(ds_encoded['test'])
print(f"Test Acc.: {test_acc*100:.2f}%")

100%|██████████| 1563/1563 [02:47<00:00,  9.34it/s]

Test Acc.: 90.54%


## 모델 저장

In [ ]:
# trainer.push_to_hub(commit_message='Training completed! 20240110')

CommitInfo(commit_url='https://huggingface.co/metamath/kcbert-large-finetuned-nsmc/commit/6ee92a41a950777d1984badf3af50ae2202b911c', commit_message='Training completed! 20240110', commit_description='', oid='6ee92a41a950777d1984badf3af50ae2202b911c', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
# p = re.compile("\/(.+)")
# m = p.search(pretrained_mode_name)[1]

# torch.save(model.state_dict(), f"nsmc-{m}.pth")

In [ ]:
# %cd /kaggle/working

/kaggle/working


In [ ]:
# !ls

__notebook_source__.ipynb  nsmc  nsmc-kcbert-large.pth


In [ ]:
# from IPython.display import FileLink
#  FileLink(r'nsmc-kcbert-large.pth')

/kaggle/working/nsmc-kcbert-large.pth